In [18]:
import os
import pickle
from tqdm import tqdm
from math import log
from nltk import PorterStemmer
import newspaper

In [19]:
folder = 'arxiv/'

In [21]:
stemmer = PorterStemmer()
url_start = '<meta name="citation_pdf_url" content="'
url_end = '" />'
authors = []
keywords = []
authors_keywords = []
for d in tqdm(os.listdir(folder)):
    with open(folder + d) as f:
        with open(folder + d) as f1:
            htmlf=f1.read().replace('\n', '')
            for line in f.readlines():
                if url_start in line:
                    url = line[len(url_start):-len(url_end)-1]
                    article = newspaper.article.Article(url)
                    article.download(input_html=htmlf)
                    article.parse()
                    a = article.authors
                    k = article.keywords
                    authors.append(a)
                    keywords.append(k)
                    authors_keywords.append([a, k])

100%|██████████| 2911/2911 [01:22<00:00, 35.50it/s]


In [22]:
v = set()
for authors_l in authors:
    if len(authors_l) > 1:
        for a in authors_l:
            v.add(a)
len(v)

4701

In [122]:
e = []
for authors_key in authors_keywords:
    aus = authors_key[0]
    key = authors_key[1]
    for i in range(len(aus)):
        for j in range(i + 1, len(aus)):
            e.append([aus[i], aus[j], key])
len(e)

14501

In [119]:
e =list(e)
v = list(v)
import string
l = []

In [133]:
l = [26, 27, 41, 53, 85, 86, 132, 138, 142, 154, 191, 231, 236, 264, 265, 275, 299, 300, 319, 338, 381, 408, 434, 458, 466, 474, 510, 551, 589, 626, 642, 651, 677, 678, 690, 693, 721, 743, 749, 756, 772, 816, 826, 902, 913, 930, 976, 980, 981, 1019, 1020, 1031, 1037, 1052, 1085, 1102, 1108, 1149, 1163, 1175, 1178, 1275, 1277, 1330, 1355, 1407, 1425, 1541, 1577, 1585, 1587, 1627, 1686, 1693, 1747, 1754, 1773, 1774, 1786, 1787, 1802, 1804, 1833, 1934, 1967, 1992, 2014, 2021, 2041, 2051, 2126, 2136, 2153, 2155, 2166, 2168, 2182, 2197, 2213, 2223, 2245, 2268, 2285, 2306, 2327, 2385, 2395, 2453, 2469, 2487, 2557, 2588, 2594, 2620, 2640, 2653, 2712, 2715, 2717, 2718, 2761, 2770, 2884, 2928, 2969, 3038, 3045, 3089, 3106, 3152, 3155, 3195, 3245, 3327, 3399, 3433, 3453, 3465, 3516, 3527, 3533, 3536, 3548, 3552, 3571, 3572, 3588, 3592, 3634, 3641, 3658, 3685, 3701, 3741, 3757, 3765, 3770, 3818, 3822, 3860, 3868, 3909, 3953, 3957, 4012, 4170, 4213, 4253, 4270, 4294, 4305, 4321, 4343, 4358, 4372, 4401, 4459, 4470, 4497, 4594, 4603, 4639, 4641, 4653, 4657, 4659]

In [ ]:
fl = True
while (fl):
    fl = False
    with open('2.gml', 'w') as f:
        f.write('graph ['+'\n')
        for i in range(len(v)):
            if i in l:
                continue
            try:
                line = bytes(v[i], 'ascii')
            except:
                l.append(i)
                print(l)
                fl = True
            f.write('  node [' +'\n')
            f.write('    id '+ str(i) +'\n')
            f.write('    label "'+ v[i] +'"\n')
            f.write('    name "'+ v[i] +'"\n')
            f.write('  ]'+'\n')
        for i in range(len(e)):
            if v.index(e[i][0]) in l or v.index(e[i][1]) in l:
                continue
            f.write('  edge [' +'\n')
            f.write('    source '+ str(v.index(e[i][0])) +'\n')
            f.write('    target '+ str(v.index(e[i][1])) +'\n')
            f.write('    weight 1' +'\n')
            f.write('  ]'+'\n')
        f.write(']')

In [76]:
G=nx.Graph()
G.add_nodes_from(list(v))
for e_i in e:
    G.add_edge(e_i[0], e_i[1], name=e_i[2])

In [ ]:
nx.draw_networkx(
            G, pos=nx.circular_layout(G),
            node_shape='o', node_size=4000, font_size=20,
            edge_color='#555555', width=3.0
        )

In [29]:
import networkx as nx

from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4



plot = Plot(plot_width=400, plot_height=400,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Graph Interaction Demonstration"

plot.add_tools(HoverTool(tooltips=None), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: fill_color [renderer: GlyphRenderer(id='0d391509-f6a1-4699-ad6b-c5d99f29de3c', ...)]
